In [1]:
import pandas as pd

import polars as pl
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
# Correct function definition
def set_table_dtypes_pandas(df):
    # Iterate over the DataFrame's columns to identify and cast columns
    for col_name in df.columns:
        # If the last letter of the column name is 'P' or 'A', cast to float
        if col_name[-1] in ('P', 'A'):
            df[col_name] = df[col_name].astype(float)
    return df


def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip "/content/drive/MyDrive/Dataset/credit/home-credit-credit-risk-model-stability.zip" -d "/content/tmp"

Archive:  /content/drive/MyDrive/Dataset/credit/home-credit-credit-risk-model-stability.zip
  inflating: /content/tmp/csv_files/test/test_applprev_1_0.csv  
  inflating: /content/tmp/csv_files/test/test_applprev_1_1.csv  
  inflating: /content/tmp/csv_files/test/test_applprev_1_2.csv  
  inflating: /content/tmp/csv_files/test/test_applprev_2.csv  
  inflating: /content/tmp/csv_files/test/test_base.csv  
  inflating: /content/tmp/csv_files/test/test_credit_bureau_a_1_0.csv  
  inflating: /content/tmp/csv_files/test/test_credit_bureau_a_1_1.csv  
  inflating: /content/tmp/csv_files/test/test_credit_bureau_a_1_2.csv  
  inflating: /content/tmp/csv_files/test/test_credit_bureau_a_1_3.csv  
  inflating: /content/tmp/csv_files/test/test_credit_bureau_a_1_4.csv  
  inflating: /content/tmp/csv_files/test/test_credit_bureau_a_2_0.csv  
  inflating: /content/tmp/csv_files/test/test_credit_bureau_a_2_1.csv  
  inflating: /content/tmp/csv_files/test/test_credit_bureau_a_2_10.csv  
  inflating: /co

In [5]:
df_train_base = pd.read_csv("/content/tmp/csv_files/train/train_base.csv")

In [6]:
df_train_base.isnull().sum()

case_id          0
date_decision    0
MONTH            0
WEEK_NUM         0
target           0
dtype: int64

In [7]:
df_train_base.head(5)

,case_id,date_decision,MONTH,WEEK_NUM,target
0,0,2019-01-03,201901,0,0
1,1,2019-01-03,201901,0,0
2,2,2019-01-04,201901,0,0
3,3,2019-01-03,201901,0,0
4,4,2019-01-04,201901,0,1


In [8]:
df_train_base['date_decision'] = pd.to_datetime(df_train_base['date_decision'])

In [9]:
df_train_base['year'] = df_train_base['date_decision'].dt.year

df_train_base['month'] = df_train_base['date_decision'].dt.month

In [10]:
df_train_base.head(5)

,case_id,date_decision,MONTH,WEEK_NUM,target,year,month
0,0,2019-01-03,201901,0,0,2019,1
1,1,2019-01-03,201901,0,0,2019,1
2,2,2019-01-04,201901,0,0,2019,1
3,3,2019-01-03,201901,0,0,2019,1
4,4,2019-01-04,201901,0,1,2019,1


In [11]:
df_train_base['target'].value_counts()

0    1478665
1      47994
Name: target, dtype: int64

In [12]:
df_train_base.describe()

,case_id,MONTH,WEEK_NUM,target,year,month
count,1.526659e+06,1.526659e+06,1.526659e+06,1.526659e+06,1.526659e+06,1.526659e+06
mean,1.286077e+06,2.019363e+05,4.076904e+01,3.143728e-02,2.019299e+03,6.425584e+00
std,7.189466e+05,4.473597e+01,2.379798e+01,1.744964e-01,4.576547e-01,3.512803e+00
min,0.000000e+00,2.019010e+05,0.000000e+00,0.000000e+00,2.019000e+03,1.000000e+00
25%,7.661975e+05,2.019060e+05,2.300000e+01,0.000000e+00,2.019000e+03,3.000000e+00
50%,1.357358e+06,2.019100e+05,4.000000e+01,0.000000e+00,2.019000e+03,7.000000e+00
75%,1.739022e+06,2.020010e+05,5.500000e+01,0.000000e+00,2.020000e+03,9.000000e+00
max,2.703454e+06,2.020100e+05,9.100000e+01,1.000000e+00,2.020000e+03,1.200000e+01


In [13]:
df_train_appl_prev = pd.read_csv("/content/tmp/csv_files/train/train_applprev_1_0.csv")

<ipython-input-13-dcfca2790d5b>:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train_appl_prev = pd.read_csv("/content/tmp/csv_files/train/train_applprev_1_0.csv")


In [14]:
train_static_0_0 = pd.read_csv("/content/tmp/csv_files/train/train_static_0_0.csv")
train_static_0_1 = pd.read_csv("/content/tmp/csv_files/train/train_static_0_1.csv")





<ipython-input-14-8080af57ca84>:1: DtypeWarning: Columns (20,45,46,53,57,84,143,146,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_static_0_0 = pd.read_csv("/content/tmp/csv_files/train/train_static_0_0.csv")
<ipython-input-14-8080af57ca84>:2: DtypeWarning: Columns (20,45,46,56,57,84,143,146,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_static_0_1 = pd.read_csv("/content/tmp/csv_files/train/train_static_0_1.csv")


In [15]:
train_static_combined = pd.concat([train_static_0_0, train_static_0_1], ignore_index=True)


In [16]:
train_static_combined = set_table_dtypes_pandas(train_static_combined)

In [17]:
train_static_cb = pd.read_csv("/content/tmp/csv_files/train/train_static_cb_0.csv",low_memory=False)
train_static_cb = set_table_dtypes_pandas(train_static_cb)

train_person_1 = pd.read_csv("/content/tmp/csv_files/train/train_person_1.csv",low_memory=False)
train_person_1 = set_table_dtypes_pandas(train_person_1)

train_credit_bureau_b_2 = pd.read_csv("/content/tmp/csv_files/train/train_credit_bureau_b_2.csv",low_memory=False)
train_credit_bureau_b_2 = set_table_dtypes_pandas(train_credit_bureau_b_2)

In [18]:
import pandas as pd
df_test_base = pd.read_csv("/content/tmp/csv_files/test/test_base.csv",low_memory=False)

df_test_base = set_table_dtypes_pandas(df_test_base)


test_static_0_0 = pd.read_csv("/content/tmp/csv_files/test/test_static_0_0.csv",low_memory=False)


test_static_0_0 = set_table_dtypes_pandas(test_static_0_0)




test_static_cb = pd.read_csv("/content/tmp/csv_files/test/test_static_cb_0.csv",low_memory=False)
test_static_cb = set_table_dtypes_pandas(test_static_cb)

test_person_1 = pd.read_csv("/content/tmp/csv_files/test/test_person_1.csv",low_memory=False)
test_person_1 = set_table_dtypes_pandas(test_person_1)

test_credit_bureau_b_2 = pd.read_csv("/content/tmp/csv_files/test/test_credit_bureau_b_2.csv",low_memory=False)
test_credit_bureau_b_2 = set_table_dtypes_pandas(test_credit_bureau_b_2)

In [19]:
# Assuming test_person_1 is your pandas DataFrame

# Step 1: Select columns (this step is actually implicit in the following steps in pandas)
# Step 2: Filter rows where 'num_group1' == 0
filtered_df = test_person_1[test_person_1['num_group1'] == 0]

# Step 3: Drop the 'num_group1' column
filtered_df = filtered_df.drop(columns=['num_group1'])

# Step 4: Rename 'housetype_905L' to 'person_housetype'
test_person_1_feats_2 = filtered_df.rename(columns={'housetype_905L': 'person_housetype'})


In [20]:
import pandas as pd

# Assuming test_credit_bureau_b_2 is your pandas DataFrame

# Group by 'case_id' and aggregate
test_credit_bureau_b_2_feats = test_credit_bureau_b_2.groupby('case_id').agg(
    pmts_pmtsoverdue_635A_max=('pmts_pmtsoverdue_635A', 'max'),
    # Use a lambda function for the conditional aggregation
    pmts_dpdvalue_108P_over31=(
        'pmts_dpdvalue_108P',
        lambda x: (x > 31).astype(int).max()
    )
).reset_index()

# Resetting the index is optional, depending on whether you want 'case_id' as a column or as an index


In [21]:
# Assuming train_static and train_static_cb are your DataFrames
selected_static_cols = [col for col in train_static_combined.columns if col[-1] in ("A", "M")]
print(selected_static_cols)

selected_static_cb_cols = [col for col in train_static_cb.columns if col[-1] in ("A", "M")]
print(selected_static_cb_cols)


['amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'disbursedcredamount_1113A', 'downpmt_116A', 'inittransactionamount_650A', 'lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M', 'lastapprcredamount_781A', 'lastcancelreason_561M', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcommoditycat_161M', 'lastrejectcommodtypec_5251769M', 'lastrejectcredamount_222A', 'lastrejectreason_759M', 'lastrejectreasonclient_4145040M', 'maininc_215A', 'maxannuity_159A', 'maxannuity_4075009A', 'maxdebt4_972A', 'maxinstallast24m_3658928A', 'maxlnamtstart6m_4525199A', 'maxoutstandbalancel12m_4187113A', 'maxpmtlast3m_4525190A', 'previouscontdistrict_112M', 'price_1097A', 'sumoutstandtotal_3546847A', 'sumoutstandtotalest_4493215A', 'totaldebt_9A', 'totalsettled_863A', 'totinstallas

In [ ]:
# Merge train_basetable with train_static on "case_id", selecting specific columns from train_static
data = pd.merge(df_train_base,
                train_static_combined[["case_id"] + selected_static_cols],
                on="case_id",
                how="left")

# Merge the resulting DataFrame with train_static_cb on "case_id", selecting specific columns from train_static_cb
data = pd.merge(data,
                train_static_cb[["case_id"] + selected_static_cb_cols],
                on="case_id",
                how="left")

# Continue joining with other DataFrames
data = pd.merge(data, train_person_1, on="case_id", how="left")
data = pd.merge(data, train_person_1, on="case_id", how="left")
data = pd.merge(data, train_credit_bureau_b_2, on="case_id", how="left")


In [ ]:
import pandas as pd

# Correcting the aggregation operation
test_person_1_agg = test_person_1.groupby('case_id').agg({
    'mainoccupationinc_384A': 'max',
    # For the custom aggregation, first convert the condition to an integer inside the agg function
}).reset_index()

# Add the custom aggregation as a separate step
test_person_1_agg['mainoccupationinc_384A_any_selfemployed'] = test_person_1[test_person_1['incometype_1044T'] == 'SELFEMPLOYED'].groupby('case_id')['incometype_1044T'].apply(lambda x: 1 if not x.empty else 0).reindex(test_person_1_agg['case_id']).fillna(0).astype(int)




In [ ]:
# Assuming test_credit_bureau_b_2 is a pandas DataFrame

# Correctly perform the grouping and aggregation
test_credit_bureau_b_2_feats = test_credit_bureau_b_2.groupby('case_id').agg({
    'pmts_pmtsoverdue_635A': 'max',
    # You cannot directly use a lambda like this for the condition; it needs to be handled differently
}).reset_index()

# To apply the condition, you need to do it outside the agg method
# First, create a mask for your condition
condition_mask = test_credit_bureau_b_2['pmts_dpdvalue_108P'] > 31

# Apply the condition, group by 'case_id', and then aggregate with max
test_credit_bureau_b_2['pmts_dpdvalue_108P_over31'] = (condition_mask).astype(int)
dpd_over_31_max = test_credit_bureau_b_2.groupby('case_id')['pmts_dpdvalue_108P_over31'].max().reset_index()

# Merge this result back into your aggregated DataFrame
test_credit_bureau_b_2_feats = test_credit_bureau_b_2_feats.merge(dpd_over_31_max, on='case_id', how='left')



In [ ]:
# Merging all the tables together on 'case_id'
data_submission = df_train_base.merge(
    test_static_0_0[['case_id']+selected_static_cols], how='left', on='case_id'
).merge(
    test_static_cb[['case_id']+selected_static_cb_cols], how='left', on='case_id'
).merge(
    test_person_1_agg, how='left', on='case_id'
).merge(
    test_person_1_feats_2, how='left', on='case_id'
).merge(
    test_credit_bureau_b_2_feats, how='left', on='case_id'
)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming 'data' is a pandas DataFrame and has been loaded

# Shuffling and splitting case IDs
case_ids = data["case_id"].unique()
np.random.seed(1)  # Set seed for reproducibility
np.random.shuffle(case_ids)
case_ids_train, case_ids_temp = train_test_split(case_ids, train_size=0.6, random_state=1)
case_ids_valid, case_ids_test = train_test_split(case_ids_temp, train_size=0.5, random_state=1)

# Selecting columns based on naming conventions
cols_pred = [col for col in data.columns if col[-1].isupper() and col[:-1].islower()]
print(cols_pred)

# Function to filter data based on case_ids and split into base, features, and target
def from_case_ids_to_datasets(case_ids_subset):
    filtered_data = data[data["case_id"].isin(case_ids_subset)]
    base = filtered_data[["case_id", "WEEK_NUM", "target"]]
    X = filtered_data[cols_pred]
    y = filtered_data["target"]
    return base, X, y

# Splitting data into train, validation, and test sets
base_train, X_train, y_train = from_case_ids_to_datasets(case_ids_train)
base_valid, X_valid, y_valid = from_case_ids_to_datasets(case_ids_valid)
base_test, X_test, y_test = from_case_ids_to_datasets(case_ids_test)

# Convert columns in X_* DataFrames from strings to categories
def convert_strings(df):
    for col in df.columns:
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("category")
    return df

# Applying the conversion to each DataFrame
X_train = convert_strings(X_train)
X_valid = convert_strings(X_valid)
X_test = convert_strings(X_test)


In [ ]:
print(f"Train: {X_train.shape}")
print(f"Valid: {X_valid.shape}")
print(f"Test: {X_test.shape}")

In [ ]:
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_valid = lgb.Dataset(X_valid, label=y_valid, reference=lgb_train)

params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 3,
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "n_estimators": 1000,
    "verbose": -1,
}

gbm = lgb.train(
    params,
    lgb_train,
    valid_sets=lgb_valid,
    callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10)]
)

In [ ]:
for base, X in [(base_train, X_train), (base_valid, X_valid), (base_test, X_test)]:
    y_pred = gbm.predict(X, num_iteration=gbm.best_iteration)
    base["score"] = y_pred

print(f'The AUC score on the train set is: {roc_auc_score(base_train["target"], base_train["score"])}')
print(f'The AUC score on the valid set is: {roc_auc_score(base_valid["target"], base_valid["score"])}')
print(f'The AUC score on the test set is: {roc_auc_score(base_test["target"], base_test["score"])}')